# Text Mining K-Means

In [6]:


#***************************************************************************************
# 1. Cargamos el corpus de textos
#***************************************************************************************

print("\n\n1. Cargamos el corpus")
trainCorpus = ["Me gustan las vacas",
               "Me gustan los caballos",
               "odio los perros",
               "odio los caballos",
               "me gustan las ranas",
               "me gusta el helado",
               "no quiero comer",
               "los helados son cremosos"]
print (trainCorpus)







#*********************************************************************************************************
# 2. Vectorizamos los textos del corpus (convertimos cada texto en un vector de frecuencias de palabras)
#*********************************************************************************************************
print("\n\n2. Vectorizamos los textos")


from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()


#Transformamos los documentos en una matriz de tf's de documentos.
vectorizer.fit(trainCorpus) #El vectorizador aprende el vocabulario del corpus
print ("Atributos:",vectorizer.get_feature_names())
tfMatrix = vectorizer.transform(trainCorpus) #Extraemos las frecuencias de palabras (tf)
print ("Matriz tf:\n",tfMatrix.toarray())


#La matriz tf es nuestro dataset, donde:
# - cada fila representa una muestra (un documento del corpus)
# - cada columna representa un atributo (la frecuencia de una palabra en dicho documento)
print("nº de muestras: %d, nº de atributos: %d" % tfMatrix.shape)






#***************************************************************************************
# 3. Clusterizamos los documentos mediante el algoritmo K-means
#***************************************************************************************
print("\n\n3. Clusterizamos los textos")

from sklearn.cluster import KMeans
from sklearn import metrics

#Asignamos a kmeans un valor de k=2, es decir que el algoritmo intentará encontrar 2 clusters
k=2
km = KMeans(n_clusters=k,max_iter=100)
km.fit(tfMatrix)
print ("Clusters:",km.labels_)
for i in range(k):
    print ("\nCluster",i,":")
    for j in range(km.labels_.size):
        if km.labels_[j]==i:
            print ("\t",trainCorpus[j])



#***************************************************************************************
# 4. Medimos la calidad de nuestro cluster
#***************************************************************************************
print("\n\n4. Medimos la calidad de nuestro cluster")

#Ground truth son las categorias correctas puestas a mano (1= texto positivo, 0=negativo), para comparar con las automaticas de kmeans
groundTruth = [1,1,0,0,1,1,0,1] 
print ("Clusters:    ",km.labels_)
print ("Ground Truth:",groundTruth)


#Un cluster es homogéneo si todos sus elementos contienen miembros de una misma clase
print("Homogeneity: %0.3f" % metrics.homogeneity_score(groundTruth, km.labels_))

#Una clase es completa si todos sus elementos pertenecen al mismo cluster
print("Completeness: %0.3f" % metrics.completeness_score(groundTruth, km.labels_))

print("V-measure: %0.3f" % metrics.v_measure_score(groundTruth, km.labels_))




#***************************************************************************************
# 5. Usamos los clusters previos para clasificar un nuevo texto entrante
#***************************************************************************************
print("\n\n5. Clasificamos un nuevo texto entrante")

testCorpus = ["odio los animales"]
tfMatrixTest =  vectorizer.transform(testCorpus)
print ("Nuevo texto:",testCorpus)
print ("Nueva matriz tf:",tfMatrixTest.toarray() )

prediction = km.predict(tfMatrixTest)[0]
print ("Prediccion: Cluster",prediction)












1. Cargamos el corpus
['Me gustan las vacas', 'Me gustan los caballos', 'odio los perros', 'odio los caballos', 'me gustan las ranas', 'me gusta el helado', 'no quiero comer', 'los helados son cremosos']


2. Vectorizamos los textos
Atributos: ['caballos', 'comer', 'cremosos', 'el', 'gusta', 'gustan', 'helado', 'helados', 'las', 'los', 'me', 'no', 'odio', 'perros', 'quiero', 'ranas', 'son', 'vacas']
Matriz tf:
 [[0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1]
 [1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0]]
nº de muestras: 8, nº de atributos: 18


3. Clusterizamos los textos
Clusters: [1 1 0 0 1 1 0 0]

Cluster 0 :
	 odio los perros
	 odio los caballos
	 no quiero comer
	 los helados son cremosos

Cluster 1 :
	 Me gustan las vacas
	 Me gustan los caballos
	 me gustan las r

In [7]:
# Vamos a jugar con el código y a realizar la práctica:

#***************************************************************************************
# 1. Cargamos el corpus de textos
#***************************************************************************************

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

categories = ['atheism','space']
trainCorpus = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=42)

print("%d documents" % len(trainCorpus.data))
print("%d categories" % len(trainCorpus.target_names))
print()

groundTruth = trainCorpus.target
true_k = np.unique(groundTruth).shape[0]


#*********************************************************************************************************
# 2. Vectorizamos los textos del corpus (convertimos cada texto en un vector de frecuencias de palabras)
#*********************************************************************************************************
print("\n\n2. Vectorizamos los textos")


from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.6, max_features=1000,
                                 min_df=2, stop_words='english',
                                 use_idf=True)


#Transformamos los documentos en una matriz de Tfidf's de documentos.
vectorizer.fit(trainCorpus) #El vectorizador aprende el vocabulario del corpus
print ("Atributos:",vectorizer.get_feature_names())
tfidfMatrix = vectorizer.transform(trainCorpus) #Extraemos las frecuencias de palabras (tf)
print ("Matriz tf:\n",tfidfMatrix.toarray())


#La matriz tf es nuestro dataset, donde:
# - cada fila representa una muestra (un documento del corpus)
# - cada columna representa un atributo (la frecuencia de una palabra en dicho documento)
print("nº de muestras: %d, nº de atributos: %d" % tfidfMatrix.shape)






#***************************************************************************************
# 3. Clusterizamos los documentos mediante el algoritmo K-means
#***************************************************************************************
print("\n\n3. Clusterizamos los textos")

from sklearn.cluster import KMeans
from sklearn import metrics

#Asignamos a kmeans un valor de k=2, es decir que el algoritmo intentará encontrar 2 clusters
k=2
km = KMeans(n_clusters=k,max_iter=100)
km.fit(tfidfMatrix)
print ("Clusters:",km.labels_)
for i in range(k):
    print ("\nCluster",i,":")
    for j in range(km.labels_.size):
        if km.labels_[j]==i:
            print ("\t",trainCorpus[j])



#***************************************************************************************
# 4. Medimos la calidad de nuestro cluster
#***************************************************************************************
print("\n\n4. Medimos la calidad de nuestro cluster")

#Ground truth son las categorias correctas puestas a mano (1= texto positivo, 0=negativo), para comparar con las automaticas de kmeans
print ("Clusters:    ",km.labels_)
print ("Ground Truth:",groundTruth)


#Un cluster es homogéneo si todos sus elementos contienen miembros de una misma clase
print("Homogeneity: %0.3f" % metrics.homogeneity_score(groundTruth, km.labels_))

#Una clase es completa si todos sus elementos pertenecen al mismo cluster
print("Completeness: %0.3f" % metrics.completeness_score(groundTruth, km.labels_))

print("V-measure: %0.3f" % metrics.v_measure_score(groundTruth, km.labels_))




#***************************************************************************************
# 5. Usamos los clusters previos para clasificar un nuevo texto entrante
#***************************************************************************************
print("\n\n5. Clasificamos un nuevo texto entrante")

testCorpus = ["odio los animales"]
tfMatrixTest =  vectorizer.transform(testCorpus)
print ("Nuevo texto:",testCorpus)
print ("Nueva matriz tf:",tfMatrixTest.toarray() )

prediction = km.predict(tfMatrixTest)[0]
print ("Prediccion: Cluster",prediction)



URLError: <urlopen error [Errno -2] Name or service not known>